In [187]:
import sys
import pandas as pd
import numpy as np
from sklearn.pipeline import make_pipeline
from sklearn.preprocessing import PowerTransformer
from sklearn.compose import make_column_transformer, make_column_selector
from sklearn.cluster import KMeans

sys.path.append("/Users/nwong/Workspace/Projects/tdt4173_project/src")

from feature_engineering.sklearn_transformers import *
from feature_engineering.store_features import *
from feature_engineering.bus_stop_features import *
from feature_engineering.utils import *
import h2o
from h2o.automl import H2OAutoML

stop_importance_levels = ['Mangler viktighetsnivå',
                          'Standard holdeplass',
                          'Lokalt knutepunkt',
                          'Nasjonalt knutepunkt',
                          'Regionalt knutepunkt',
                          'Annen viktig holdeplass']
store_types = ['lv1_desc', 'lv2_desc', 'lv3_desc', 'lv4_desc']
geo_groups = ['grunnkrets_id','t_district','municipality_name']

In [188]:
stores_train_df = set_year_2016(pd.read_csv("/Users/nwong/Workspace/Projects/tdt4173_project/data/raw/stores_train.csv"))
stores_test_df = set_year_2016(pd.read_csv("/Users/nwong/Workspace/Projects/tdt4173_project/data/raw/stores_test.csv"))
stores_extra_df = set_year_2016(pd.read_csv("/Users/nwong/Workspace/Projects/tdt4173_project/data/raw/stores_extra.csv"))

income_dist = set_year_2016(pd.read_csv("/Users/nwong/Workspace/Projects/tdt4173_project/data/raw/grunnkrets_income_households.csv"))
age_dist = set_year_2016(pd.read_csv("/Users/nwong/Workspace/Projects/tdt4173_project/data/raw/grunnkrets_age_distribution.csv"))
household_dist = set_year_2016(pd.read_csv("/Users/nwong/Workspace/Projects/tdt4173_project/data/raw/grunnkrets_households_num_persons.csv"))
grunnkrets_df = combine_keys(set_year_2016(pd.read_csv("/Users/nwong/Workspace/Projects/tdt4173_project/data/raw/grunnkrets_norway_stripped.csv")))
plaace_df = pd.read_csv("/Users/nwong/Workspace/Projects/tdt4173_project/data/raw/plaace_hierarchy.csv")

bus_stops_df = bus_stops_lat_lon(pd.read_csv("/Users/nwong/Workspace/Projects/tdt4173_project/data/raw/busstops_norway.csv"))


/Users/nwong/Workspace/Projects/tdt4173_project/src/feature_engineering/bus_stop_features.py:11: FutureWarning: In a future version of pandas all arguments of StringMethods.split except for the argument 'pat' will be keyword-only.
  bus_stops_df[['lon', 'lat']] = bus_stops_df['lng_lat'].str.split(


In [189]:
stores_train_merged = encode_levels(stores_train_df.merge(plaace_df, on="plaace_hierarchy_id", how="left"))
stores_test_merged = encode_levels(stores_test_df.merge(plaace_df, on="plaace_hierarchy_id", how="left"))
stores_extra_merged = encode_levels(stores_extra_df.merge(plaace_df, on="plaace_hierarchy_id", how="left"))

In [190]:
compare_train_df = pd.concat([stores_train_merged, stores_extra_merged], ignore_index=True)
compare_test_df = pd.concat([stores_test_merged, stores_extra_merged], ignore_index=True)

In [191]:
def stores_in_radius_new(stores_merged, compare_df, radius=0.05):
    lv_1 = stores_in_radius(stores_merged, compare_df, radius=radius, store_type_group='lv1_desc')
    lv_2 = stores_in_radius(stores_merged, compare_df, radius=radius, store_type_group='lv2_desc')
    lv_3 = stores_in_radius(stores_merged, compare_df, radius=radius, store_type_group='lv3_desc')
    lv_4 = stores_in_radius(stores_merged, compare_df, radius=radius, store_type_group='lv4_desc')
    all_count = stores_in_radius(stores_merged, compare_df, radius=radius, store_type_group=None)
    
    return lv_1.merge(lv_2, on="store_id", how="inner").merge(lv_3, on="store_id", how="inner").merge(lv_4, on="store_id", how="inner").merge(all_count, on="store_id", how="inner")

In [192]:
def new_pop_density(stores_df, age_dist, grunnkrets_df, geo_groups):
    population_density = population_density_grouped_by_geo_group(stores_df, age_dist, grunnkrets_df, geo_groups)
    return population_density.fillna(population_density.mean())

In [193]:

def new_age_dist(stores_df, age_df, grunnkrets_df, geo_groups):
    _age_dist = age_dist_by_geo_group(stores_train_df, age_dist, grunnkrets_df, geo_groups)
    return _age_dist.fillna(_age_dist.mean())

In [194]:
cluster_simil = ClusterSimilarity(n_clusters=100, gamma=1., random_state=42, sample_weight=stores_train_df[['revenue']])
similarities = cluster_simil.fit(stores_train_df[["lat", "lon"]])

def new_clustering(cluster_simil, stores_df):
    return pd.DataFrame(cluster_simil.transform(stores_df[['lat', 'lon']]), columns=cluster_simil.get_feature_names_out(), index=stores_df.store_id).reset_index()

In [197]:
kmeans = KMeans(n_clusters=1200, init='k-means++')
kmeans.fit(stores_train_df[['lat', 'lon']], sample_weight=stores_train_df['revenue'])

def new_kmeans_weighted(dataframe):
    dataframe['cluster_label'] = kmeans.predict(dataframe[['lat', 'lon']])
    dataframe['cluster_label_str'] = dataframe['cluster_label'].astype(str) + '_cluster'
    dataframe.drop('cluster_label', axis=1, inplace=True)


In [198]:
# new_kmeans_weighted(stores_train_df)
# new_kmeans_weighted(stores_test_df)

In [199]:
def distance_to_closest_group(stores_df, compare_df, group):
    """
        Mall or chain
    """
    mat = cdist(
        stores_df[['lat', 'lon']],
        compare_df[compare_df[group].notna()][['lat', 'lon']], metric="euclidean"
    )
    
    new_df = pd.DataFrame(
        mat, index=stores_df['store_id'], columns=compare_df[compare_df[group].notna()]['store_id']
    )
    
    new_df = new_df[new_df > 0]
    
    stores = new_df.index
    # closest = new_df.idxmin(axis=1)
    distance = new_df.min(axis=1)
    
    return pd.DataFrame({'store_id': stores.values, f'distance_closest_{group}': distance.values})

In [200]:
merged_stores_train = stores_train_merged \
    .merge(grunnkrets_df, on="grunnkrets_id", how="left") \
    .merge(income_dist, on="grunnkrets_id", how="left") \
    .merge(store_closest_by_store_groups(stores_train_merged, compare_train_df, store_types), on="store_id", how="left") \
    .merge(stores_in_radius_new(stores_train_merged, compare_train_df), on="store_id", how="left") \
    .merge(new_clustering(cluster_simil, stores_train_df), on="store_id", how="left") \
    .merge(new_pop_density(stores_train_df, age_dist, grunnkrets_df, geo_groups), on="store_id", how="left") \
    .merge(average_household_income_by_geo_groups(stores_train_df, geo_groups, income_dist, household_dist, grunnkrets_df)) \
    .merge(bus_stops_distance_by_importance(stores_train_df, bus_stops_df, stop_importance_levels).reset_index(level=0), on="store_id", how="left") \
    .merge(new_age_dist(stores_train_df, age_dist, grunnkrets_df, geo_groups), on="store_id", how="left") \
    .merge(distance_to_closest_group(stores_train_merged, compare_train_df, "mall_name"), on="store_id", how="left") \
    .merge(distance_to_closest_group(stores_train_merged, compare_train_df, "chain_name"), on="store_id", how="left") 
        
merged_stores_test = stores_test_merged \
    .merge(grunnkrets_df, on="grunnkrets_id", how="left") \
    .merge(income_dist, on="grunnkrets_id", how="left") \
    .merge(store_closest_by_store_groups(stores_test_merged, compare_test_df, store_types), on="store_id", how="left") \
    .merge(stores_in_radius_new(stores_test_merged, compare_test_df), on="store_id", how="left") \
    .merge(new_clustering(cluster_simil, stores_test_df), on="store_id", how="left") \
    .merge(new_pop_density(stores_test_df, age_dist, grunnkrets_df, geo_groups), on="store_id", how="left") \
    .merge(average_household_income_by_geo_groups(stores_test_df, geo_groups, income_dist, household_dist, grunnkrets_df)) \
    .merge(bus_stops_distance_by_importance(stores_test_df, bus_stops_df, stop_importance_levels).reset_index(level=0), on="store_id", how="left") \
    .merge(new_age_dist(stores_test_df, age_dist, grunnkrets_df, geo_groups), on="store_id", how="left") \
    .merge(distance_to_closest_group(stores_test_merged, compare_test_df, "mall_name"), on="store_id", how="left") \
    .merge(distance_to_closest_group(stores_test_merged, compare_test_df, "chain_name"), on="store_id", how="left") 

/var/folders/z1/l03w8mpn5xz3mghrk0j2w5gr0000gn/T/ipykernel_71538/272456682.py:3: FutureWarning: The default value of numeric_only in DataFrame.mean is deprecated. In a future version, it will default to False. In addition, specifying 'numeric_only=None' is deprecated. Select only valid columns or specify the value of numeric_only to silence this warning.
  return population_density.fillna(population_density.mean())
/var/folders/z1/l03w8mpn5xz3mghrk0j2w5gr0000gn/T/ipykernel_71538/1837615021.py:3: FutureWarning: The default value of numeric_only in DataFrame.mean is deprecated. In a future version, it will default to False. In addition, specifying 'numeric_only=None' is deprecated. Select only valid columns or specify the value of numeric_only to silence this warning.
  return _age_dist.fillna(_age_dist.mean())
/var/folders/z1/l03w8mpn5xz3mghrk0j2w5gr0000gn/T/ipykernel_71538/272456682.py:3: FutureWarning: The default value of numeric_only in DataFrame.mean is deprecated. In a future vers

In [201]:
target_labels = merged_stores_train[['store_id', 'revenue']].copy()
merged_stores_train = merged_stores_train.copy().drop('revenue', axis=1)

In [202]:
pd.DataFrame(merged_stores_train.columns).to_csv("features.csv")

In [223]:
merged_stores_train['store_name_first'] = merged_stores_train['store_name'].str.split(' ').str[0]
merged_stores_train['store_name_last'] = merged_stores_train['store_name'].str.split(' ').str[-1]
merged_stores_train['address_first'] = merged_stores_train['address'].str.extract('(\D+)\s+(\d+)\s?(.*)')[0] + merged_stores_train['municipality_name']

merged_stores_test['store_name_first'] = merged_stores_test['store_name'].str.split(' ').str[0]
merged_stores_test['store_name_last'] = merged_stores_test['store_name'].str.split(' ').str[-1]
merged_stores_test['address_first'] = merged_stores_test['address'].str.extract('(\D+)\s+(\d+)\s?(.*)')[0] + merged_stores_test['municipality_name']

In [259]:
merged_stores_train['is_chain'] = merged_stores_train['chain_name'].isna()
merged_stores_train['is_mall'] = merged_stores_train['mall_name'].isna()

merged_stores_test['is_chain'] = merged_stores_test['chain_name'].isna()
merged_stores_test['is_mall'] = merged_stores_test['mall_name'].isna()

In [260]:
bus_col_names = [
    'distance_to_mangler_viktighetsnivå',
    'distance_to_standard_holdeplass',
    'distance_to_lokalt_knutepunkt',
    'distance_to_nasjonalt_knutepunkt',
    'distance_to_regionalt_knutepunkt',
    'distance_to_annen_viktig_holdeplass'
]
merged_stores_train["bus_distance_to_store"] = merged_stores_train[bus_col_names].min(axis = 1)
merged_stores_test["bus_distance_to_store"] = merged_stores_train[bus_col_names].min(axis = 1)

In [269]:
inc_cols = [
    'store_id',
    # 'store_name_first',
    # 'store_name_first',
    # 'store_name_last',
    'address_first',
    
    # 'plaace_hierarchy_id',
    # 'chain_name',
    'is_chain',
    # 'mall_name',
    'is_mall',
    'sales_channel_name_x',
    
    'grunnkrets_id',
    'municipality_name',
    't_district',
    # 'address',
    
    'lv1',
    'level2',
    'level3',
    'lv4',
]

yeo_cols = [
    'all_households',
    'avg_household_income_t_district',
    'avg_household_income_municipality_name',
    't_district_density',
    # 'municipality_name_density',
    
    'all_stores_in_radius',
    # 'lv1_desc_in_radius',
    'lv2_desc_in_radius',
    'lv3_desc_in_radius',
    # 'lv4_desc_in_radius',
]

box_cols = [
    'area_km2',
    # 'distance_to_lv1_desc',
    'distance_to_lv2_desc',
    'distance_to_lv3_desc',
    # 'distance_to_lv4_desc',
    # 'grunnkrets_id_density',
    # 't_district_density',
    'municipality_name_density',
    'distance_to_lokalt_knutepunkt',
    'distance_to_regionalt_knutepunkt',
    'distance_to_annen_viktig_holdeplass',
    'distance_to_nasjonalt_knutepunkt',
    'distance_closest_mall_name',
    'distance_closest_chain_name',
    'bus_distance_to_store',
]

_merged_stores_train = merged_stores_train.filter(inc_cols+yeo_cols+box_cols)
_merged_stores_test = merged_stores_test.filter(inc_cols+yeo_cols+box_cols)

In [270]:
_merged_stores_train['grunnkrets_id'] = _merged_stores_train['grunnkrets_id'].astype('str')

In [271]:
_merged_stores_test['grunnkrets_id'] = _merged_stores_test['grunnkrets_id'].astype('str')

In [273]:
yeo_pipeline = make_pipeline(
    SimpleImputer(strategy="mean"),
    PowerTransformer()
)
box_pipeline = make_pipeline(
    SimpleImputer(strategy="mean"),
    PowerTransformer(method="box-cox")
)

preprocessing = make_column_transformer(
        (yeo_pipeline, yeo_cols),
        (box_pipeline, box_cols),
        remainder="passthrough"
    )

def new_transformer(merged_stores_df, preprocessing):
    return pd.DataFrame(preprocessing.transform(merged_stores_df), columns=preprocessing.get_feature_names_out(), index=merged_stores_df.index)

In [274]:
preprocessing.fit(_merged_stores_train)

_merged_stores_train = new_transformer(_merged_stores_train, preprocessing)
_merged_stores_test = new_transformer(_merged_stores_test, preprocessing)

In [275]:
pt = PowerTransformer()
rev_transformed = pt.fit_transform(target_labels[["revenue"]])
_merged_stores_train["revenue"] = rev_transformed

In [276]:
# _merged_stores_train["z_score"] = (_merged_stores_train.revenue - _merged_stores_train.revenue.mean()) / _merged_stores_train.revenue.std()
# train_no_outliers = (_merged_stores_train[(_merged_stores_train.z_score > -2) & (_merged_stores_train.z_score < 2)])

# _merged_stores_train.drop(['z_score'], axis=1, inplace=True)

In [277]:
# Start the H2O cluster (locally)
h2o.init()

# Import a sample binary outcome train/test set into H2O
train = h2o.H2OFrame(_merged_stores_train)
test = h2o.H2OFrame(_merged_stores_test)

cat_vars = inc_cols

cat_vars = [f'remainder__{i}' for i in cat_vars]

for cat in cat_vars:
    train[cat] = train[cat].asfactor()
    test[cat] = test[cat].asfactor()


#test = h2o.H2OFrame(pd.concat([test_set, stores_test_enriched[['store_id']]], axis=1).drop(drop_cols, axis=1))

Checking whether there is an H2O instance running at http://localhost:54321 . connected.


H2O_cluster_uptime:,5 hours 37 mins
H2O_cluster_timezone:,Europe/Oslo
H2O_data_parsing_timezone:,UTC
H2O_cluster_version:,3.38.0.2
H2O_cluster_version_age:,16 days
H2O_cluster_name:,H2O_from_python_nwong_l2zh49
H2O_cluster_total_nodes:,1
H2O_cluster_free_memory:,1.251 Gb
H2O_cluster_total_cores:,16
H2O_cluster_allowed_cores:,16
H2O_cluster_status:,"locked, healthy"


Parse progress: |████████████████████████████████████████████████████████████████| (done) 100%
Parse progress: |████████████████████████████████████████████████████████████████| (done) 100%


In [278]:
# Identify predictors and response
x = train.columns
y = "revenue"
x.remove(y)

# Run AutoML for 20 base models
aml = H2OAutoML(max_models=20, seed=1, exclude_algos=['deeplearning'])
aml.train(x=x, y=y, training_frame=train)

AutoML progress: |███████████████████████████████████████████████████████████████| (done) 100%


,mean,sd,cv_1_valid,cv_2_valid,cv_3_valid,cv_4_valid,cv_5_valid
mae,0.5753306,0.0048882,0.5743651,0.5757141,0.5710888,0.5834376,0.5720471
mean_residual_deviance,0.5566410,0.0113249,0.549559,0.5599357,0.5574877,0.5730606,0.5431621
mse,0.5566410,0.0113249,0.549559,0.5599357,0.5574877,0.5730606,0.5431621
null_deviance,2572.4175,88.38749,2661.035,2536.0876,2643.515,2579.3167,2442.133
r2,0.4429477,0.0147948,0.4587953,0.4290717,0.4470122,0.4258385,0.4540207
residual_deviance,1431.8708,57.912815,1439.8445,1446.874,1461.7327,1479.0693,1331.8335
rmse,0.746053,0.0075792,0.7413225,0.7482886,0.7466509,0.7570077,0.7369953
rmsle,nan,0.0,nan,nan,nan,nan,nan


In [283]:
lb = aml.leaderboard
lb.head(rows=lb.nrows)  # Print all rows instead of default (10 rows)

model_id,rmse,mse,mae,rmsle,mean_residual_deviance
StackedEnsemble_AllModels_1_AutoML_11_20221112_173832,0.746149,0.556739,0.575229,nan,0.556739
StackedEnsemble_BestOfFamily_1_AutoML_11_20221112_173832,0.746651,0.557487,0.575479,nan,0.557487
GBM_grid_1_AutoML_11_20221112_173832_model_3,0.759076,0.576196,0.585708,nan,0.576196
XRT_1_AutoML_11_20221112_173832,0.759087,0.576214,0.595082,nan,0.576214
GBM_grid_1_AutoML_11_20221112_173832_model_2,0.765576,0.586106,0.591661,nan,0.586106
GBM_2_AutoML_11_20221112_173832,0.767457,0.58899,0.59222,nan,0.58899
GBM_1_AutoML_11_20221112_173832,0.770174,0.593169,0.594564,nan,0.593169
DRF_1_AutoML_11_20221112_173832,0.77024,0.59327,0.591443,nan,0.59327
GBM_5_AutoML_11_20221112_173832,0.771329,0.594949,0.596138,nan,0.594949
GBM_grid_1_AutoML_11_20221112_173832_model_1,0.773501,0.598304,0.598007,nan,0.598304


In [284]:
# The leader model is stored here
aml.leader

,mean,sd,cv_1_valid,cv_2_valid,cv_3_valid,cv_4_valid,cv_5_valid
mae,0.5753306,0.0048882,0.5743651,0.5757141,0.5710888,0.5834376,0.5720471
mean_residual_deviance,0.5566410,0.0113249,0.549559,0.5599357,0.5574877,0.5730606,0.5431621
mse,0.5566410,0.0113249,0.549559,0.5599357,0.5574877,0.5730606,0.5431621
null_deviance,2572.4175,88.38749,2661.035,2536.0876,2643.515,2579.3167,2442.133
r2,0.4429477,0.0147948,0.4587953,0.4290717,0.4470122,0.4258385,0.4540207
residual_deviance,1431.8708,57.912815,1439.8445,1446.874,1461.7327,1479.0693,1331.8335
rmse,0.746053,0.0075792,0.7413225,0.7482886,0.7466509,0.7570077,0.7369953
rmsle,nan,0.0,nan,nan,nan,nan,nan


In [285]:
m = h2o.get_model(lb[3,"model_id"])
m.varimp(use_pandas=True)

,variable,relative_importance,scaled_importance,percentage
0,remainder__municipality_name,170220.859375,1.000000,0.185585
1,remainder__lv4,170220.046875,0.999995,0.185584
2,remainder__level3,142048.484375,0.834495,0.154869
3,remainder__sales_channel_name_x,141828.562500,0.833203,0.154630
4,remainder__is_chain,73944.593750,0.434404,0.080619
5,remainder__level2,41467.160156,0.243608,0.045210
6,remainder__store_id,27863.406250,0.163690,0.030378
7,remainder__t_district,17693.496094,0.103944,0.019290
8,remainder__grunnkrets_id,10880.461914,0.063920,0.011863
9,remainder__lv1,10435.583984,0.061306,0.011377


In [286]:
preds_avg = aml.predict(test)
preds_best = aml.leader.predict(test)

stackedensemble prediction progress: |███████████████████████████████████████████| (done) 100%
stackedensemble prediction progress: |

/Users/nwong/opt/anaconda3/envs/sklearn-env/lib/python3.10/site-packages/h2o/job.py:83: UserWarning: Test/Validation dataset column 'remainder__store_id' has levels not trained on: ["810130822-971657243-219275", "810182482-971658150-11075", "811480282-971683597-179256", "811604372-988862983-86893", "811618772-912015351-623892", "811625612-911659778-433802", "811661112-911677962-612425", "811665762-971687568-21394", "811672602-974079658-32805", "811673862-974995875-40151", ...8557 not listed..., "999649262-990192235-115914", "999649440-972257710-39400", "999653987-911550016-614706", "999655866-811552712-614673", "999657117-971794577-1512", "999662072-911698862-612468", "999665624-992886544-338735", "999666124--87921-87921", "999666728-911570688-517908", "999668615-980238296-585056"]
  warnings.warn(w)
/Users/nwong/opt/anaconda3/envs/sklearn-env/lib/python3.10/site-packages/h2o/job.py:83: UserWarning: Test/Validation dataset column 'remainder__address_first' has levels not trained on: ["

███████████████████████████████████████████| (done) 100%


In [287]:
df = test.cbind(preds_best)
df = df.as_data_frame(use_pandas=True)
result = df.loc[:,("remainder__store_id", 'predict')]
submission = result.rename(columns = {"remainder__store_id": "id",  "predict" : "predicted"})
submission['predicted'] = pt.inverse_transform(submission[['predicted']])
submission.to_csv("StackedEnsembleBestOfFamily13.csv", index = False)

submission

/Users/nwong/opt/anaconda3/envs/sklearn-env/lib/python3.10/site-packages/sklearn/base.py:493: FutureWarning: The feature names should match those that were passed during fit. Starting version 1.2, an error will be raised.
Feature names unseen at fit time:
- predicted
Feature names seen at fit time, yet now missing:
- revenue

  warnings.warn(message, FutureWarning)


,id,predicted
0,914206820-914239427-717245,4.901588
1,916789157-916823770-824309,8.670136
2,913341082-977479363-2948,8.267403
3,889682582-889697172-28720,8.601211
4,997991699-998006945-417222,8.948124
...,...,...
8572,917323003-917383529-844309,4.164916
8573,917353379-917411824-845904,3.288336
8574,917072302-917089248-833647,6.463330
8575,916960557-916993161-829908,2.730550
